<a href="https://colab.research.google.com/github/Haj1h0/llm-from-scratch-code/blob/main/ch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tiktoken

In [2]:
from importlib.metadata import version

print("파이토치 버전:", version("torch"))
print("tiktoken 버전:", version("tiktoken"))

파이토치 버전: 2.9.0+cu126
tiktoken 버전: 0.12.0


In [3]:
import os
import requests

if not os.path.exists("the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("총 문자 개수:", len(raw_text))
print(raw_text[:99])

총 문자 개수: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [6]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [7]:
# 각 항목에서 공백을 삭제하고 빈 문자열을 제외합니다.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [8]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [9]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [10]:
print(len(preprocessed))

4690


In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [12]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [13]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [14]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab # ex) {"hello": 0, "world": 1, ",": 2, "": 3}
        self.int_to_str = {i:s for s,i in vocab.items()} # {i:s ...} : 뒤집어서 (숫자: 문자열) 형태로 저장

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text) # 'hello,. world'
        # 정규식 패턴에 해당하는 문자 기준으로 문장을 split함. 괄호 없이 쓰면 구분자가 사라지는데 패턴을 ()로 감싸면 split 결과 안에 구분자도 포함됨
        # ["Hello", ",", " ", "world", "!", ""]

        preprocessed = [
            item.strip() for item in preprocessed if item.strip() # item.strip() = 앞뒤 공백을 제거, 빈 문자열이면 제외 → 공백 같은 것들은 다 사라짐
        ]
        ids = [self.str_to_int[s] for s in preprocessed] # 토큰 리스트 → vocab을 이용해 숫자 리스트로 변환
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 구둣점 문자 앞의 공백을 삭제합니다.
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

# encode 함수는 텍스트를 토큰 ID로 바꿉니다.
# decode 함수는 토큰 ID를 텍스트로 바꿉니다.

In [15]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [16]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [17]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [18]:
# 특수 문맥 토큰 추가하기
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
# 알지 못하는 단어를 표현하는 "<|unk|>"
# 훈련에서 텍스트의 끝을 나타내기 위해 사용된 "<|endoftext|>"

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
len(vocab.items())

1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [21]:
# 새로운 <unk> 토큰을 사용할 때와 방법을 알려 주기 위한 토크나이저 수정
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int  else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 구둣점 문자 앞의 공백을 삭제합니다.
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [22]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
tokenizer.decode(tokenizer.encode(text))
# <unk>는 모르는 여러 단어가 와도 다 동일한 하나의 토큰, decode 시 원래 단어를 복원할 방법 0%, 그래서 modern LLM은 <unk> 없는 방식으로 tokenizer 설계함

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

In [25]:
# 바이트 페어 인코딩
# 어휘사전에 없는 단어를 더 작은 부분단어나 개별 문자로 분할하여 처리
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [26]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
# tiktoken은 기본적으로 special token이 텍스트에 직접 적혀 있으면 오류를 발생
# 특수 토큰(special token)을 “토큰 그대로 허용해서 인식해라”는 설정

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [27]:
tokenizer.special_tokens_set

{'<|endoftext|>'}

In [28]:
print(tokenizer.encode(text, allowed_special='all'))
# 모든 special token을 텍스트 안에서 허용하겠다.

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [29]:
strings = tokenizer.decode(integers)

print(strings)
# BPE 토크나이저는 알지 못하는 단어를 부분단어나 개별 문자로 분할

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [30]:
# 슬라이딩 윈도로 데이터 샘플링하기
# 한 번에 한 단어씩 생성하도록 LLM을 훈련하므로 시퀀스에 있는 다음 단어가 예측할 타깃이 되도록 훈련 데이터를 준비
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [31]:
enc_sample = enc_text[50:]

In [32]:
# context length가 제한되어 있어서 일정 토큰 길이로 작게 잘라서 학습하고 이때 잘라낸 한 덩어리를 청크라 부른다.
# text chunk = 모델이 한 번에 볼 수 있는 입력 토큰들의 덩어리

context_size = 4 # 청크 크기 = 4개 토큰

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1] # window size = 1

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [33]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [35]:
# 모델 학습에 쓸 데이터를 잘라서, 정해진 크기(batch)로, 반복적으로 모델에게 공급해주는 도구 = 데이터 로더
import torch
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride): # max_length: 모델이 한 번에 볼 수 있는 context size, stride: 슬라이딩 윈도우가 몇 칸씩 이동할지
        self.input_ids = [] # x(batch의 입력)
        self.target_ids = [] # y(next-token target)
        # 두 리스트에 슬라이딩 윈도우로 나눈 데이터가 계속 append됨.

        # 전체 텍스트를 토큰화합니다.
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "토큰화된 입력의 개수는 적어도 max_length+1과 같아야 합니다."
        # 최소한 max_length보다 토큰이 많아야 context window를 만들 수 있으니까 체크한다. assert가 실패하면 메시지 출력 후 코드 중단.

        # 슬라이딩 윈도를 사용해 책을 max_length 길이의 중첩된 시퀀스로 나눕니다.
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1] # 타깃(y)은 항상 입력(x)보다 “1칸 오른쪽으로만” 이동한다. stride는 “다음 청크의 시작 위치”를 결정하는 것이지, 타깃이 몇 칸 이동하는 것과는 전혀 상관없다.
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            # 텐서로 바꿔 리스트에 저장

    def __len__(self): # Dataset의 크기
        return len(self.input_ids) # input_ids 개수 == chunk 개수, 즉, 몇 개의 (x, y)쌍이 있는지 반환

    def __getitem__(self, idx): # 특정 index의 sample을 반환
        return self.input_ids[idx], self.target_ids[idx]

# stride 때문에 chunk가 서로 겹치는 것은 매우 정상이고, GPT 학습에서는 오히려 필수적이다.
# 모델이 더 많은 “위치 기반 문맥 패턴”을 배울 수 있기 때문이다.
# 보통 GPT 학습에서는 stride = 1 또는 GPU 메모리 때문에 최소 stride = max_length/2 정도

In [36]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,  stride=128, shuffle=True, drop_last=True, num_workers=0):
# tokenizer 생성, dataset 생성, dataloader 생성

    # 토크나이저를 초기화합니다.
    tokenizer = tiktoken.get_encoding("gpt2")

    # 데이터셋을 만듭니다.
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 데이터 로더를 만듭니다.
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [37]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [40]:
# 문맥 크기 4, 배치 크기 1인 데이터 로더
dataloader = create_dataloader_v1( raw_text, batch_size=2, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
# 예: batch_size=1
# → (input_tensor 1개, target_tensor 1개)
#예: batch_size=4
#→ (input_tensor 4개, target_tensor 4개)
#즉, (x, y) 튜플이 넘어옴.

print(first_batch)

[tensor([[  40,  367, 2885, 1464],
        [ 367, 2885, 1464, 1807]]), tensor([[ 367, 2885, 1464, 1807],
        [2885, 1464, 1807, 3619]])]


In [41]:
second_batch = next(data_iter)
print(second_batch)
# stride :	chunks 생성 단계, 텍스트에서 chunk를 얼마나 겹치며 만들지 결정, stride는 “윈도우 이동 거리”
# batch_size : DataLoader가 모델에 전달할 때 이미 만들어진 chunks 중 몇 개를 한 번에 묶어서 모델에 넣을지, 모델에 한 번에 몇 개의 chunk를 넣을 것인가?
# ex) batch_size = 2, (chunk1, chunk2) 묶어서 학습, (chunk3, chunk4) 묶어서 학습

[tensor([[2885, 1464, 1807, 3619],
        [1464, 1807, 3619,  402]]), tensor([[1464, 1807, 3619,  402],
        [1807, 3619,  402,  271]])]


In [42]:
# 배치 간에 중첩이 있으면 과대적합이 증가될 수 있으므로 스트라이드를 증가시킵니다.
# 큰 stride는 학습 속도도 훨씬 빠르다.
# 대부분의 파인튜닝은 “독립적인 instruction-respond” 쌍이라 겹칠 이유가 없다
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("입력:\n", inputs)
print("\n타깃:\n", targets)

# Instruction 파인튜닝에서는 데이터가 이미 완성된 독립 샘플이므로, 슬라이딩 윈도우로 겹치는 chunk를 만드는 것은 의미 없는 중복 연산이며, 오히려 데이터 구조를 파괴하는 잘못된 방식이다.
# → 그래서 stride = max_length 가 표준.
# Instruction Fine-Tuning은 pretrained language model을 사용자의 명령(instruction)을 수행하는 assistant-style 모델로 바꾸기 위해 Instruction–Response 쌍으로 supervised fine-tuning 하는 과정이다.

입력:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

타깃:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [43]:
# 토큰 임베딩 만들기
input_ids = torch.tensor([2, 3, 5, 1]) # (토큰화를 거친 후) 토큰 ID가 2, 3, 5, 1인 샘플을 가정
vocab_size = 6 # 어휘사전에는 6개 단어만 들어 있고, 임베딩 크기는 3이라고 가정
output_dim = 3

torch.manual_seed(123) # PyTorch는 가중치를 랜덤으로 초기화하는데 seed를 고정하면 항상 같은 랜덤값 → 재현성(reproducibility)을 위해 필수
embedding_layer = torch.nn.Embedding(vocab_size, output_dim) # LLM에서 단어 ID → Dense 벡터로 변환하는 층

In [44]:
print(embedding_layer.weight)
# embedding_layer.weight는 “전체 Embedding 행렬”을 출력한다.
# 6x3 가중치 행렬

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [45]:
print(embedding_layer(torch.tensor([3])))
# embedding_layer 가중치 행렬에 있는 네 번째 행

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [46]:
print(embedding_layer(input_ids))
# 룩업 연산 진행한 임베딩 층 결과

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [47]:
# 단어 위치 인코딩하기

# 바이트 페어 인코더의 어휘 사전 크기는 50,257입니다.
# 입력 토큰을 256차원의 벡터 표현으로 인코딩한다고 가정합니다.
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [48]:
# 데이터 로더에서 데이터를 샘플링한 다음 배치에 있는 각 샘플의 토큰을 256차원 벡터로 임베딩할 수 있습니다.
# 배치 크기가 8이고 샘플마다 4개의 토큰이 있다면 8 x 4 x 256 크기의 텐서가 만들어 집니다.
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [49]:
print("토큰 ID:\n", inputs)
print("\n입력 크기:\n", inputs.shape)

토큰 ID:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

입력 크기:
 torch.Size([8, 4])


In [50]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# 임베딩 벡터의 값을 확인합니다.
print(token_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584, -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,  0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,  1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,  0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,  0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473]],

        ...,

        [[-1.6156,  0.9610, -2.6437,  ..., -0.9645,  1.0888,  1.6383],
         [-0.3985, -0.9235, -1.31

In [51]:
# GPT-2는 절대 위치 임베딩을 사용하므로 또 다른 임베딩 층을 만들면 된다.
# GPT-2 스타일 절대 위치 임베딩(position embedding)”을 직접 만드는 코드
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# 임베딩 층의 가중치를 확인합니다.
print(pos_embedding_layer.weight)

Parameter containing:
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       requires_grad=True)


In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# 위치 임베딩 값을 확인합니다.
print(pos_embeddings)

torch.Size([4, 256])
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)


In [53]:
# LLM에 사용될 입력 임베딩을 만들기 위해 토큰 임베딩과 위치 임베딩을 더한다.
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# 입력 임베딩 값을 확인합니다.
print(input_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 2.2288,  0.5619,  0.8286,  ..., -0.6272, -0.2987,  0.8900],
         [ 2.0903, -0.4664, -0.0593,  ...,  0.9115, -1.0493, -1.6473],
         [-0.7158, -0.8304,  1.2494,  ...,  2.3952,  1.8773,  0.8051],
         [ 0.2703,  0.4029,  3.0514,  ...,  0.3595, -1.4548,  0.8310]],

        [[ 3.2835,  1.1749, -1.4150,  ..., -0.3281,  2.4332,  0.6924],
         [-0.2199, -0.9114, -0.1750,  ...,  1.5337, -0.1998,  0.1462],
         [ 1.5197, -1.4240,  0.4391,  ...,  1.0494, -1.4318,  2.3057],
         [ 0.2893,  0.8346, -0.1884,  ...,  1.9602,  0.8709,  0.8796]],

        [[ 0.9662,  0.0952, -0.4640,  ..., -1.0320,  1.6290,  1.7771],
         [ 2.4468, -0.2154,  1.4984,  ...,  1.8766,  0.5595, -0.1423],
         [-0.3856, -2.5393,  1.1556,  ...,  3.6157,  1.3267,  0.4944],
         [-0.2487, -0.5275,  2.0009,  ...,  0.2930,  0.5977,  1.3300]],

        ...,

        [[ 0.1219,  0.3991, -3.2740,  ..., -1.1921,  2.6637,  2.6728],
         [ 1.2438, -1.6436, -1.11